In [72]:
import pandas as pd
import re

In [43]:
df = pd.read_csv("/content/TM Bot v1.0 - utterances.csv")

In [24]:
df.sample(20)

,,Author,Type,Utterance,Intent,Entities,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9
58,NaN,NaN,NaN,I want to go <country>,"recommend_activity, recommend_poi",NaN,NaN,NaN,NaN,NaN
154,NaN,NaN,NaN,Do you pick up at the hotel? We are staying in...,tour_inquiry,"hotel, location",NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,I want tickets for <keyword>.,book_tour,<keyword>,NaN,NaN,NaN,NaN
35,NaN,NaN,NaN,What is the contact info?,needs_assistance,NaN,NaN,NaN,NaN,NaN
136,NaN,NaN,NaN,is the <keyword> on <date> available other tim...,tour_available,"keyword, date, time",NaN,NaN,NaN,NaN
113,NaN,NaN,NaN,"Hello, is it possible to pay extra for a seat ...",restaurant_inquiry,NaN,NaN,NaN,NaN,NaN
170,NaN,NaN,NaN,Is lunch included in tour?,tour_inquiry,NaN,NaN,NaN,NaN,NaN
97,NaN,NaN,NaN,How soon will I find out if the reservation is...,reservation_regulations,NaN,NaN,NaN,NaN,NaN
192,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
32,NaN,NaN,NaN,Can I talk to someone?,needs_assistance,NaN,NaN,NaN,NaN,NaN


In [45]:
df =  df[df['Entities'].notna()]

In [37]:
df.drop(columns=["Unnamed: 6", "Unnamed: 7", "Unnamed: 8", "Unnamed: 9"], inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [21]:
entities = set()
for x in df["Entities"]:
  x = x.split(',')
  entities.update(x)

In [22]:
entities

{' ',
 ' <tour_time><keyword>',
 ' available',
 ' available date',
 ' budget',
 ' date',
 ' hotel name',
 ' hotel pickup',
 ' instruction',
 ' keyword',
 ' language',
 ' location',
 ' location to location',
 ' number',
 ' number of people',
 ' number_of_people',
 ' request',
 ' time',
 '*',
 '<budget>',
 '<keyword>',
 '<location>',
 '<number_of_people>',
 "We aren't supporting this",
 'assistance',
 'category',
 'hotel',
 'keyword',
 'location',
 'number',
 'number_of_people',
 'people',
 'restaurant',
 'time',
 'tour',
 'tour_parameter: departure time',
 'transportation',
 'update'}

Keywordfiller takes in an utterance and the key to be filled, along with a list of values to fill it, and returns a list of all the versions of the utterance filled with items from the list

In [173]:
def keyWordFiller(utterance, keyword, inp_vals):
  #print(utterance, keyword, inp_vals)
  if keyword not in utterance:
    print("ERROR, DNE", keyword)
    return
  keyword = "<" + keyword + ">"
  temp = utterance.split(keyword)
  total = []
  if len(temp) == 2:
    partA, partB = temp
    total = [partA + x + partB for x in inp_vals]
  else:
    total = [temp[0] + x for x in inp_vals]
  return total

This would be implemented as a database rather than a dictionary in a larger scale project

In [178]:
inps = {}
inps["budget"] = ["2", "twenty five dollars", "under a thousand"]
inps["location"] = ["Russia", "Poland", "San Fransisco"]
inps["tour_time"] = ["around 3 pm", "at about 2", "before noon"]
inps["number_of_people"] = ["3 people", "1", "a few"]
inps["keyword"] = ["KEYWORD"]
inps["time"] = ["around 3", "noon", "7:30pm"]
inps["date"] = ["Friday March 7", "10-12-15"]
inps["transportation"] = ["Jeep", "car"]
inps["duration"] = ["7 nights", "overnight", "a couple days"]
inps["hotel"] = ["Mariott", "nice hotel", "a hotel with a pool"]

I decided to test this keyword filler using only a subset of the data due to time.  I filled entities for hotel, location, time, and hotel.

In [214]:
df = pd.read_csv("/content/TM Bot v1.0 - utterances.csv")
df =  df[df['Entities'].notna()]
df.drop(columns=["Unnamed: 6", "Unnamed: 7", "Unnamed: 8", "Unnamed: 9"], inplace=True)
df = df[df.Entities.str.contains("budget|location|tour_time", regex=True)]

final = []
for i, row in df[["Utterance", "Intent", "Entities"]].iterrows():
  utt = [row["Utterance"]]
  while (utt is not None and utt != []):
    temp = utt.pop(0)
    if "<" in temp:
      ent = re.findall(r"\<([A-Za-z0-9_]+)\>", temp)
      rows = keyWordFiller(temp, ent[0], inps[ent[0]])
      if utt != [] and utt is not None:
        utt.extend(rows)
      else:
        utt = rows
    else:
      final.append((temp, row["Intent"]))
finaldf = pd.DataFrame(final, columns=["Utterance", "Intent"])
finaldf

,Utterance,Intent
0,I wanna buy tickets for KEYWORD around 3 for 3...,book_tour
1,I wanna buy tickets for KEYWORD around 3 for 1.,book_tour
2,I wanna buy tickets for KEYWORD around 3 for a...,book_tour
3,I wanna buy tickets for KEYWORD noon for 3 peo...,book_tour
4,I wanna buy tickets for KEYWORD noon for 1.,book_tour
...,...,...
170,Does KEYWORD offer pickup at Poland?,tour_inquiry
171,Does KEYWORD offer pickup at San Fransisco?,tour_inquiry
172,Can a tour guide pick us up at Russia?,tour_inquiry
173,Can a tour guide pick us up at Poland?,tour_inquiry


In [215]:
finaldf.to_csv("filled_data.csv", index = False, header=True)

In [216]:
final

[('I wanna buy tickets for KEYWORD around 3 for 3 people.', 'book_tour'),
 ('I wanna buy tickets for KEYWORD around 3 for 1.', 'book_tour'),
 ('I wanna buy tickets for KEYWORD around 3 for a few.', 'book_tour'),
 ('I wanna buy tickets for KEYWORD noon for 3 people.', 'book_tour'),
 ('I wanna buy tickets for KEYWORD noon for 1.', 'book_tour'),
 ('I wanna buy tickets for KEYWORD noon for a few.', 'book_tour'),
 ('I wanna buy tickets for KEYWORD 7:30pm for 3 people.', 'book_tour'),
 ('I wanna buy tickets for KEYWORD 7:30pm for 1.', 'book_tour'),
 ('I wanna buy tickets for KEYWORD 7:30pm for a few.', 'book_tour'),
 ('Tell me more about Russia.', 'describe_location, recommend_poi'),
 ('Tell me more about Poland.', 'describe_location, recommend_poi'),
 ('Tell me more about San Fransisco.', 'describe_location, recommend_poi'),
 ('Show me top attractions Russia.', 'describe_location, recommend_poi'),
 ('Show me top attractions Poland.', 'describe_location, recommend_poi'),
 ('Show me top attra